In [1]:
import os
import glob
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: /opt/ros/kinetic/lib/python2.7/dist-packages/cv2.so: undefined symbol: PyCObject_Type

### Loading the data stored in Data/Videos/Evaluation

In [ ]:
#To get all the video files in the given evaluation directory
file_list = []
pwd = os.getcwd()
video_dir = 'Data/Videos/Evaluation'
video_path = os.path.join(pwd,video_dir)
file_list = [os.path.join(video_path,file) for file in os.listdir(os.path.join(pwd, video_path)) \
             if file.endswith(".avi")]
len(file_list)
# to get a particular frame use cap.set(cv.CAP_PROP_POS_FRAMES, index), where index is the frame number you want
# have to use cap.read() to get the image after setting the frame number

### Function to calculate abs_diff and return the best image for further processing

In [ ]:
def get_final_image(cap, image_list):
    
#     returns the final image that will be used for detecting the dice corners and pips
    
#   total number of images that have been sampled from the video at 1 frame per second
    num_images = len(image_list)
    
    prev_diff = np.float('inf')

    if num_images > 1:
    
        for i in reversed(range(num_images-1)):
#             print(good_frames[i])
            img = cap.set(cv.CAP_PROP_POS_FRAMES, good_frames[i])
            _,img = cap.read()
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        #     print('Image 1',good_frames[i])
        #     plt.imshow(img)
        #     plt.show()

            img2 = cap.set(cv.CAP_PROP_POS_FRAMES, good_frames[i-1])
            _,img2 = cap.read()
            img2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
        #     print('Image 2', good_frames[i-1])
        #     plt.imshow(img2)
        #     plt.show()

            err = np.sum((img.astype("float") - img2.astype("float")) ** 2)
            err /= float(img.shape[0] * img.shape[1])
#             print('difference = ', err)

            if err < prev_diff:
                prev_diff = err
                img_final = img2
                image_index = good_frames[i-1]

    else:
        img_final = cap.set(cv.CAP_PROP_POS_FRAMES, image_list[0])
        _,img_final = cap.read()
        img_final = cv.cvtColor(img_final, cv.COLOR_BGR2GRAY)
        image_index = good_frames[0]
            
    return(img_final, image_index)
    

### Extract frames at 1 per second from all videos

In [ ]:
sel_images = []
image_indices = []

for vid in range(len(file_list)):
    
    print('vid num: {}'.format(vid + 1))
    
    cap = cv.VideoCapture(file_list[vid])
    fps = cap.get(cv.CAP_PROP_FPS)
    num_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    good_frames = []

#     print('frame rate: {}'.format(fps))

    while(cap.isOpened):
        # Capture frame-by-frame
        ret, frame = cap.read()
    
        if (ret):
            # Our operations on the frame come here
            frame_index = cap.get(cv.CAP_PROP_POS_FRAMES)
            
            if (frame_index % int(fps) == 0):
#                 print("frame_id:{}".format(frame_index))
                good_frames.append(frame_index)
                    
            if cv.waitKey(int(fps)) & 0xFF == ord('q'):
                break
            
        else:
#             print(good_frames)
#             print('len: {}'.format(len(good_frames)))
           
            processed_img, index = get_final_image(cap, good_frames)
#             plt.imshow(processed_img)
#             plt.show()
            image_indices.append(index)
            sel_images.append(processed_img)
            break
            
        
cap.release()
cv.destroyAllWindows()

In [ ]:
image_dir = 'Data/Images'
image_path = os.path.join(pwd,image_dir)

image_list = [os.path.join(image_path,file) for file in os.listdir(os.path.join(pwd, image_path)) \
             if file.endswith(".jpg")]

In [2]:

for i in range(len(sel_images)):
    print(i+1)
#     plt.imshow(sel_images[i])
#     plt.show()
    cv.imwrite(os.path.join(image_path , 'video_{}.jpg'.format(i)), sel_images[i])

NameError: name 'sel_images' is not defined

In [3]:
for img_id in image_list:
    print(img_id)
    gray = cv.imread(img_id, 0)
#     base_img = cv.cvtColor(gray,cv.COLOR_GRAY2RGB)
    gray = cv.medianBlur(gray,7)
#     should try it once with the adaptive thresholding
#     gray = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv.THRESH_BINARY,11,2)
    edges = cv.Canny(gray, 100, 200);
    im2, contours, hierarchy = cv.findContours(edges,\
                                                cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        area = cv.contourArea(cnt)
        if area < 50:
#             im2 = cv.drawContours(gray, contours, -1, (255,0,0), 3)
            x,y,w,h = cv.boundingRect(cnt)
            im2 = cv.rectangle(gray,(x,y),(x+w,y+h),(0,255,0),2)
            

    cv.imshow('original', im2)
    k = cv.waitKey(0) & 0xFF
    if k == 27:         # wait for ESC key to exit
        cv.destroyAllWindows()  
#     cv2.imshow('canny', edges)
#     k = cv2.waitKey(0) & 0xFF
#     if k == 27:         # wait for ESC key to exit
#         cv.destroyAllWindows()  


NameError: name 'image_list' is not defined

In [4]:
image_indices

NameError: name 'image_indices' is not defined

In [5]:
#cross checking if the image indexes match the obtained frames
cap = cv.VideoCapture(file_list[3])
image_test = cap.set(cv.CAP_PROP_POS_FRAMES, image_indices[3])
_,image_test = cap.read()
image_test = cv.cvtColor(image_test, cv.COLOR_BGR2GRAY)
plt.imshow(image_test)
plt.show()


NameError: name 'cv' is not defined

In [6]:
gray = cv.imread(image_list[3])
#     base_img = cv.cvtColor(gray,cv.COLOR_GRAY2RGB)
gray = cv.medianBlur(gray,9)
#     should try it once with the adaptive thresholding
#     gray = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv.THRESH_BINARY,11,2)
edges = cv.Canny(gray, 70, 90);
# edges = cv.dilate(edges, None, iterations=1)
# edges = cv.erode(edges, None, iterations=1)
im2, contours, hierarchy = cv.findContours(edges,\
                                            cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    area = cv.contourArea(cnt)
    print(area)
    if area > 100 and area < 500:
        x,y,w,h = cv.boundingRect(cnt)
#         print(w,h, 'w,h')
        im2 = cv.rectangle(gray,(x,y),(x+w,y+h),(0,0,255),2)


cv.imshow('original', im2)
k = cv.waitKey(0) & 0xFF
if k == 27:         # wait for ESC key to exit
    cv.destroyAllWindows()  

NameError: name 'cv' is not defined